In [1]:
import os
os.environ["VLLM_USE_V1"] = "0"      # ← remove this after the bug of vllm is fixed

from enum import Enum
from pydantic import BaseModel, Field
from typing import List
import json
from string import Template

from vllm import LLM, SamplingParams
from vllm.sampling_params import GuidedDecodingParams
from vllm.lora.request import LoRARequest
from transformers import AutoTokenizer

INFO 05-02 10:54:15 [__init__.py:239] Automatically detected platform cuda.


In [2]:
# Few-shot examples
samples = [
    {
        "input": (
            "慢性関節リウマチの診断と管理\n\n"
            + "関節の腫脹、疼痛、可動域制限などの症状が認められる。"
            + "血清リウマトイド因子やCRPの上昇が典型的な検査所見として報告されている。"
            + "進行例では骨侵食や関節変形を合併し、日常生活動作に支障をきたすことがある。"
                  ),
        "output": {
            "results": [
                {
                "disease_text": "慢性関節リウマチ",
                "abnormal_findings_caused_by_the_disease": [
                    {
                    "finding_text": "関節の腫脹",
                    "finding_type": "symptom"
                    },
                    {
                    "finding_text": "疼痛",
                    "finding_type": "symptom"
                    },
                    {
                    "finding_text": "可動域制限",
                    "finding_type": "symptom"
                    },
                    {
                    "finding_text": "血清リウマトイド因子の上昇",
                    "finding_type": "examination_result"
                    },
                    {
                    "finding_text": "CRPの上昇",
                    "finding_type": "examination_result"
                    },
                    {
                    "finding_text": "骨侵食",
                    "finding_type": "complication"
                    },
                    {
                    "finding_text": "関節変形",
                    "finding_type": "complication"
                    }
                ]
                }
            ]
        }
    },
    {
        "input": (
            "糖尿病の病態\n\n"
            + "多尿、口渇、体重減少が一般的な症状として現れる。"
            + "血糖値の上昇やHbA1cの増加が認められる。"
            + "長期的には網膜症や神経障害を合併する。"
                  ),
        "output": {
            "results": [
                {
                "disease_text": "糖尿病",
                "abnormal_findings_caused_by_the_disease": [
                    {
                    "finding_text": "多尿",
                    "finding_type": "symptom"
                    },
                    {
                    "finding_text": "口渇",
                    "finding_type": "symptom"
                    },
                    {
                    "finding_text": "体重減少",
                    "finding_type": "symptom"
                    },
                    {
                    "finding_text": "血糖値の上昇",
                    "finding_type": "examination_result"
                    },
                    {
                    "finding_text": "HbA1cの増加",
                    "finding_type": "examination_result"
                    },
                    {
                    "finding_text": "網膜症",
                    "finding_type": "complication"
                    },
                    {
                    "finding_text": "神経障害",
                    "finding_type": "complication"
                    }
                ]
                }
            ]
            }
    }
]



In [3]:
# Define the schema for the output
class FindingCategory(str, Enum):
    symptom = "symptom"
    examination_result = "examination_result"
    complication = "complication"

class AbnormalFinding(BaseModel):
    finding_text: str = Field(
        ...,
        title="所見テキスト",
        description="記事中に記載された症状・検査所見・合併症の名称"
    )
    finding_type: FindingCategory = Field(
        ...,
        title="所見タイプ",
        description="`symptom`／`examination_result`／`complication` のいずれか"
    )

class DiseaseProperty(BaseModel):
    disease_text: str = Field(
        ...,
        title="疾患テキスト",
        description="抽出された疾患名の正式名称"
    )
    abnormal_findings_caused_by_the_disease: List[AbnormalFinding] = Field(
        ...,
        title="異常所見一覧",
        description="当該疾患に関連する全所見"
    )

class ExtractionResult(BaseModel):
    results: List[DiseaseProperty] = Field(
        ...,
        title="抽出結果リスト",
        description="複数疾患対応の抽出結果配列"
    )

    class Config:
        populate_by_name = True
        json_schema_extra = {
            "example": samples[0]["output"]
        }

In [4]:
prompts = {
    "system": (
        "あなたは日本語医学文献から疾患と関連所見を抽出する専門エンジンです。"
        "以下の JSON Schema に**完全準拠**し、他のテキストを一切含めず JSON のみで出力してください：\n"
        + json.dumps(ExtractionResult.model_json_schema(), ensure_ascii=False)
    ),
    "user": Template((
        " 以下の<<<INPUT>>>と<<<END>>>で囲まれたテキストを処理し、システムプロンプトで定義したJSON Schemaに"
        "**完全に従って**JSONのみを返してください。\n"
        "処理の内容は、「記事中に記載された個々の症状・検査所見・合併症の表現をそのまま抽出して記載すること」です。\n"
        "与えられたテキストに出現する『疾患テキスト』と『所見テキスト』を抽出して、そのまま記載してください。\n"
        "【所見タイプの定義】\n"
        "symptom: 自覚症状・他覚所見に加えて、医師が視診・触診・聴診などの手や簡単な器具を使って確認できる理学所見や体温・脈拍・呼吸数・血圧などのバイタルサイン、尿量や身長・体重など、簡便に確認可能な所見\n"
        "examination_result: 血液検査、X線、超音波、病理検査など、検査オーダーが必要な客観的所見\n"
        "complication: その疾患の進行・長期化で発生しうる合併症のこと。\n"
        "### タスクの対象:\n"
        "<<<INPUT>>>\n"
        "${ArticleText}\n"
        "<<<END>>>\n\n"
    ))
}

In [5]:
base_model = "unsloth/Phi-4-mini-instruct"
llm = LLM(
    model=base_model,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
    swap_space=4,
    cpu_offload_gb=0,
    enable_lora=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

INFO 05-02 10:54:17 [config.py:209] Replacing legacy 'type' key with 'rope_type'
INFO 05-02 10:54:23 [config.py:717] This model supports multiple tasks: {'embed', 'score', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 05-02 10:54:23 [arg_utils.py:1536] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
INFO 05-02 10:54:23 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5) with config: model='unsloth/Phi-4-mini-instruct', speculative_config=None, tokenizer='unsloth/Phi-4-mini-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 05-02 10:54:27 [loader.py:458] Loading weights took 1.14 seconds
INFO 05-02 10:54:27 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-02 10:54:28 [model_runner.py:1140] Model loading took 7.2370 GiB and 2.373489 seconds
INFO 05-02 10:54:38 [worker.py:287] Memory profiling takes 10.26 seconds
INFO 05-02 10:54:38 [worker.py:287] the current vLLM instance can use total_gpu_memory (47.41GiB) x gpu_memory_utilization (0.90) = 42.67GiB
INFO 05-02 10:54:38 [worker.py:287] model weights take 7.24GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 8.26GiB; the rest of the memory reserved for KV Cache is 27.11GiB.
INFO 05-02 10:54:38 [executor_base.py:112] # cuda blocks: 13882, # CPU blocks: 2048
INFO 05-02 10:54:38 [executor_base.py:117] Maximum concurrency for 131072 tokens per request: 1.69x
INFO 05-02 10:54:41 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eag

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 05-02 10:54:58 [model_runner.py:1592] Graph capturing finished in 16 secs, took 0.51 GiB
INFO 05-02 10:54:58 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 29.94 seconds


In [6]:
json_schema = ExtractionResult.model_json_schema()
guided_decoding_params_json = GuidedDecodingParams(json=json_schema)
sampling_params = SamplingParams(
                        temperature=0.0,
                        max_tokens=1024,
                        guided_decoding=guided_decoding_params_json
                        )

## Evaluate Outputs for a short sample (73 characters)

In [7]:
# Evaluate a text (samples[1]) that not used in the json schema examples.
messages = [
    {"role": "system", "content": prompts["system"]},
    {"role": "user", "content": prompts["user"].substitute(ArticleText=samples[1]["input"])}
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Phi-4-mini-instruct: Vanilla
base_outputs = llm.generate(prompts=prompt, sampling_params=sampling_params)
base_object = json.loads(base_outputs[0].outputs[0].text)
print("=== BASE MODEL ===")
display(base_object)

# Phi-4-mini-instruct with our PEFT adapter
adapter_model = "seiya/Phi-4-mini-instruct-qlora-adapter-jp-disease-finding"
lora_req = LoRARequest("our_adapter", 1, adapter_model)  # name, unique_id, path
lora_outputs = llm.generate(prompts=prompt, sampling_params=sampling_params, lora_request=lora_req)
lora_object = json.loads(lora_outputs[0].outputs[0].text)
print("=== LoRA ===")
display(lora_object)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

=== BASE MODEL ===


{'results': [{'disease_text': '糖尿病の病態',
   'abnormal_findings_caused_by_the_disease': [{'finding_text': '多尿',
     'finding_type': 'symptom'},
    {'finding_text': '口渇', 'finding_type': 'symptom'},
    {'finding_text': '体重減少', 'finding_type': 'symptom'},
    {'finding_text': '血糖値の上昇', 'finding_type': 'examination_result'},
    {'finding_text': 'HbA1cの増加', 'finding_type': 'examination_result'},
    {'finding_text': '網膜症', 'finding_type': 'complication'},
    {'finding_text': '神経障害', 'finding_type': 'complication'}]}]}

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

=== LoRA ===


{'results': [{'disease_text': '糖尿病',
   'abnormal_findings_caused_by_the_disease': [{'finding_text': '多尿',
     'finding_type': 'symptom'},
    {'finding_text': '口渇', 'finding_type': 'symptom'},
    {'finding_text': '体重減少', 'finding_type': 'symptom'},
    {'finding_text': '血糖値の上昇', 'finding_type': 'examination_result'},
    {'finding_text': 'HbA1cの増加', 'finding_type': 'examination_result'},
    {'finding_text': '網膜症', 'finding_type': 'complication'},
    {'finding_text': '神経障害', 'finding_type': 'complication'}]}]}

## Evaluate Outputs for a medium sample (1,371 characters)

In [8]:
# Review article of PseudoDisease01
article_text = (
    "__PseudoDisease01__：全身性自己免疫炎症疾患の新たな疾患概念\n\n"

    "はじめに\n"
    "__PseudoDisease01__は、近年提唱されたリウマチ・膠原病領域に属する新たな全身性自己免疫性疾患である。"
    "本疾患は特異的な自己抗体「抗Pseudo抗体」の出現を特徴とし、関節、皮膚、腎臓、神経系を中心に多臓器に炎症性病変をきたす。"
    "自己免疫反応による慢性炎症と組織線維化が病態の中核にあり、進行例では重篤な臓器障害を伴うことも少なくない。"
    "本稿では、__PseudoDisease01__の病態生理、臨床症状、検査所見、合併症、鑑別診断について概説し、今後の診断と治療の展望について述べる。\n\n"

    "病態の特徴\n"
    "__PseudoDisease01__は、異常な自己免疫応答を背景に発症する。"
    "抗Pseudo抗体の産生により、主に微小血管内皮が標的となり、炎症細胞の浸潤が誘導される。"
    "その結果、局所での組織破壊が進行し、線維芽細胞の活性化と細胞外マトリックスの沈着が亢進する。"
    "特に関節滑膜、皮膚、腎間質ではこの線維化の傾向が顕著であり、不可逆的な組織変性が多臓器機能の低下につながる。\n\n"

    "臨床症状\n"
    "症状は多彩で、慢性関節痛や朝のこわばり、遠位指節の腫脹・変形といった関節リウマチ様の症状を呈する。"
    "また、皮膚症状としては蝶形紅斑が認められ、全身倦怠感や発熱などの全身症状も頻発する。"
    "乾燥性角結膜炎や口腔乾燥といったシェーグレン症候群様の症状、多発性筋痛、末梢神経障害によるしびれやレイノー現象など、"
    "膠原病の複数の側面を併せ持つ点が本疾患の特徴である。\n\n"

    "検査所見\n"
    "血清学的には、抗Pseudo抗体の検出が診断の決め手となる。"
    "関節液分析ではリンパ球優位の炎症細胞浸潤を示し、皮膚生検では真皮の線維化と血管炎所見が認められる。"
    "腎生検では微小血管炎に加え間質線維化が確認されることが多い。"
    "MRIでは滑膜の肥厚と増殖性変化、胸部X線では肺間質の浸潤影がしばしば見られる。"
    "その他、CRPや赤沈の上昇、末梢神経伝導速度の低下、シルマー試験陽性など、臓器障害を反映する多彩な所見を示す。\n\n"

    "合併症\n"
    "本疾患の進行に伴い、肺線維症による呼吸不全や腎機能障害による慢性腎不全、心膜炎や心筋炎といった心病変を呈する例がある。"
    "また、神経障害が進行すると運動麻痺を来す可能性があり、血栓形成に伴う血管イベントも報告されている。"
    "早期診断と全身管理が不可欠である。\n\n"

    "鑑別診断\n"
    "臨床像が多彩であるため、他の自己免疫疾患との鑑別が重要となる。"
    "特に、"
    "PseudoDisease02（全身性エリテマトーデス様疾患）、"
    "PseudoDisease03（強皮症類縁疾患）、"
    "PseudoDisease04（混合性結合組織病類似疾患）、"
    "PseudoDisease05（シェーグレン症候群変異型）"
    "との鑑別が求められる。"
    "抗体プロファイルや組織所見に基づいた包括的な評価が必要である。\n\n"

    "おわりに\n"
    "__PseudoDisease01__は、複数の膠原病の臨床的特徴を併せ持ちつつ、独自の免疫学的・組織学的プロファイルを有する新たな疾患単位である。"
    "今後は、病態解明の進展とともに、バイオマーカーの確立、治療標的の同定が急務である。"
    "臨床現場では、早期発見と臓器障害の予防に重点を置いた診療体制の整備が望まれる。"
)

In [9]:
messages = [
    {"role": "system", "content": prompts["system"]},
    {"role": "user", "content": prompts["user"].substitute(ArticleText=article_text)}
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
sampling_params = SamplingParams(
                        temperature=0.0,
                        max_tokens=2048,
                        guided_decoding=guided_decoding_params_json
                        )

# Phi-4-mini-instruct: Vanilla
base_outputs = llm.generate(prompts=prompt, sampling_params=sampling_params)
base_object = json.loads(base_outputs[0].outputs[0].text)
print("=== BASE MODEL ===")
display(base_object)

# Phi-4-mini-instruct with our QLoRA adapter
adapter_model = "seiya/Phi-4-mini-instruct-qlora-adapter-jp-disease-finding"
lora_req = LoRARequest("our_adapter", 1, adapter_model)
lora_outputs = llm.generate(prompts=prompt, sampling_params=sampling_params, lora_request=lora_req)
lora_object = json.loads(lora_outputs[0].outputs[0].text)
print("=== LoRA ===")
display(lora_object)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

=== BASE MODEL ===


{'results': [{'disease_text': '__PseudoDisease01__',
   'abnormal_findings_caused_by_the_disease': [{'finding_text': '全身性自己免疫炎症疾患',
     'finding_type': 'symptom'},
    {'finding_text': '抗Pseudo抗体の産生', 'finding_type': 'examination_result'},
    {'finding_text': '関節滑膜の肥厚と増殖性変化', 'finding_type': 'examination_result'},
    {'finding_text': '肺間質の浸潤影', 'finding_type': 'examination_result'},
    {'finding_text': '肺線維症による呼吸不全', 'finding_type': 'complication'},
    {'finding_text': '心膜炎や心筋炎', 'finding_type': 'complication'},
    {'finding_text': '運動麻痺', 'finding_type': 'complication'},
    {'finding_text': '血栓形成', 'finding_type': 'complication'}]}]}

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

=== LoRA ===


{'results': [{'disease_text': '__PseudoDisease01__',
   'abnormal_findings_caused_by_the_disease': [{'finding_text': '慢性関節痛',
     'finding_type': 'symptom'},
    {'finding_text': '朝のこわばり', 'finding_type': 'symptom'},
    {'finding_text': '遠位指節の腫脹・変形', 'finding_type': 'symptom'},
    {'finding_text': '蝶形紅斑', 'finding_type': 'symptom'},
    {'finding_text': '全身倦怠感', 'finding_type': 'symptom'},
    {'finding_text': '発熱', 'finding_type': 'symptom'},
    {'finding_text': '乾燥性角結膜炎', 'finding_type': 'symptom'},
    {'finding_text': '口腔乾燥', 'finding_type': 'symptom'},
    {'finding_text': '多発性筋痛', 'finding_type': 'symptom'},
    {'finding_text': '末梢神経障害によるしびれ', 'finding_type': 'symptom'},
    {'finding_text': 'レイノー現象', 'finding_type': 'symptom'},
    {'finding_text': '抗Pseudo抗体の検出', 'finding_type': 'examination_result'},
    {'finding_text': '関節液分析でリンパ球優位の炎症細胞浸潤',
     'finding_type': 'examination_result'},
    {'finding_text': '皮膚生検で真皮の線維化と血管炎所見',
     'finding_type': 'examination_result'},
